# Text De-Toxification, part II: Building Vocabulary for condBERT
### Robert Chen, B20-AI
--------------------

## Step 0: Imports

In [1]:
import torch
import sys
import os
from pathlib import Path
import string

# Small crutch to make Jupyter see the source folder
if not sys.path.count(str(Path(os.path.realpath("")).parent)):
    sys.path.append(str(Path(os.path.realpath("")).parent))
    
import pandas as pd
import pickle
import numpy as np
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

## Step 1: Download the datasets

The **condBERT** model does not need to be trained, but we need a solid corpus for it to show acceptable results. Gladly, there are already a lot of datasets that suit this task specifically. In our case, we are going to use *Jigsaw* dataset, which was already prepared by SkolTech team and the initial *ParaNMT* dataset. 

In [2]:
#!usr/bin/bash
DATA_DIR="../data"
! bash $DATA_DIR/download_jigsaw.sh
! bash $DATA_DIR/download_paranmt.sh

Now, we need to process *ParaNMT* and *ParaDetox* datasets and add them to the existing train data.

In [19]:
paranmt_path = '../data/raw/filtered.tsv'

train_toxic = '../data/interim/condbert_vocab/train/train_toxic'
train_normal = '../data/interim/condbert_vocab/train/train_normal'

hello?


In order to process the *ParaNMT* dataset, we need to gather all texts with high toxicity scores into `train_toxic` dataset and put the rest into `train_normal` dataset. We will try to cut off at the toxicity score of 0.5. Also we will need to separate the punctuation marks with a whitespace.

In [4]:
paranmt_df = pd.read_csv(paranmt_path, sep="\t", index_col=0)
toxic_ref = paranmt_df[paranmt_df.ref_tox >= 0.5]['reference'].tolist()
toxic_trans = paranmt_df[paranmt_df.trn_tox >= 0.5]['translation'].tolist()
with open(train_toxic, 'a') as f:
    f.writelines(toxic_ref)
with open(train_toxic, 'a') as f:
    f.writelines(toxic_trans)
neutral_ref = paranmt_df[paranmt_df.ref_tox < 0.5]['reference'].tolist()
neutral_trans = paranmt_df[paranmt_df.trn_tox < 0.5]['translation'].tolist()
with open(train_normal, 'a') as f:
    f.writelines(neutral_ref)
with open(train_normal, 'a') as f:
    f.writelines(neutral_trans)
# tokenizer = WordPunctTokenizer()
# with tqdm(range(len(arr))) as t:
#     for i in t:
#         arr[i] = ' '.join(tokenizer.tokenize(arr[i]))
#         t.set_description(f'Line #{i}/{len(arr)}')
# print(arr[:5])